In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
 if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatHeader}>
          <div className={styles.chatHeaderTitle}>
            <div className={styles.iconWrapper}>
              <FluentIcons.Sparkle24Filled />
            </div>
            <Text className={styles.chatHeaderText}>{userConfig.buttons[0].label}</Text>
          </div>
          <Text className={styles.chatHeaderDescription}>{userConfig.buttons[0].description}</Text>
        </div>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => (
            <React.Fragment key={index}>
              <div
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <div dangerouslySetInnerHTML={{ __html: message.content }} />
                )}
              </div>
              {message.role === "user" && index === 0 && isFirstQuery && (
                <div className={styles.chartContainer}>
                  <StockChart data={stockData} />
                </div>
              )}
              {message.subgraphResults && (
                <div className={styles.subgraphMessage}>
                  <SubgraphResults results={message.subgraphResults} />
                </div>
              )}
            </React.Fragment>
          ))}
        </div>

In [ ]:
from typing import Dict, List
from langgraph.graph import StateGraph, END

# Define your functions
def vector_db_search(query: str) -> str:
    # Implement your vector database search here
    return f"Vector DB result for: {query}"

def query_expansion(query: str) -> List[str]:
    # Implement your query expansion logic here
    return [f"{query} expanded 1", f"{query} expanded 2"]

def api_call(query: str) -> str:
    # Implement your API call here
    return f"API result for: {query}"

# Define the routing logic
def route_query(state):
    query = state['query']
    if len(query.split()) > 3:  # Simple routing logic, adjust as needed
        return "query_expansion"
    else:
        return "vector_db_search"

# Define the query expansion and API call logic
def expand_and_call(state):
    expanded_queries = query_expansion(state['query'])
    results = []
    for eq in expanded_queries:
        results.append(api_call(eq))
    return {"expanded_results": results}

# Define the aggregation logic
def aggregate_results(state):
    if "expanded_results" in state:
        # Aggregate expanded results
        combined = " ".join(state["expanded_results"])
        return {"response": f"Aggregated results: {combined}"}
    else:
        # Return vector DB result
        return {"response": state["result"]}

# Create the graph
workflow = StateGraph(name="QueryProcessor")

# Add nodes
workflow.add_node("route", route_query)
workflow.add_node("vector_db_search", lambda state: {"result": vector_db_search(state['query'])})
workflow.add_node("expand_and_call", expand_and_call)
workflow.add_node("aggregate", aggregate_results)

# Add edges
workflow.add_edge("route", "vector_db_search")
workflow.add_edge("route", "expand_and_call")
workflow.add_edge("vector_db_search", "aggregate")
workflow.add_edge("expand_and_call", "aggregate")
workflow.add_edge("aggregate", END)

# Set the entrypoint
workflow.set_entry_point("route")

# Compile the graph
app = workflow.compile()

# Example usage
query = "What is the capital of France?"
result = app.invoke({"query": query})
print(result['response'])

In [ ]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generate multi-dimensional training data
def generate_synthetic_data(n_points=100, n_features=3):
    """
    Generate synthetic multi-dimensional data with correlations
    Returns X (features) and y (target)
    """
    torch.manual_seed(0)  # For reproducibility
    
    # Generate feature matrix
    X = torch.zeros((n_points, n_features)).to(device)
    
    # Time component (first feature)
    X[:, 0] = torch.linspace(0, 1, n_points)
    
    # Generate additional features with some correlation to time
    for i in range(1, n_features):
        X[:, i] = torch.sin(X[:, 0] * (2 * math.pi * (i + 1))) + \
                  torch.randn(n_points).to(device) * 0.1
    
    # Generate target variable (complex function of features)
    y = torch.sin(X[:, 0] * (2 * math.pi))  # Time component
    for i in range(1, n_features):
        y += 0.5 * torch.sin(X[:, i] * math.pi)  # Additional feature effects
    
    # Add noise to target
    y += torch.randn(n_points).to(device) * math.sqrt(0.04)
    
    return X, y

# Define the multi-dimensional GP Model
class MultiDimExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultiDimExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean().to(device)
        
        # Use an RBF kernel with ARD (Automatic Relevance Determination)
        base_kernel = gpytorch.kernels.RBFKernel(ard_num_dims=train_x.size(1)).to(device)
        self.covar_module = gpytorch.kernels.ScaleKernel(base_kernel).to(device)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def train_and_evaluate(n_features=3, n_points=100, initial_train_size=10):
    # Generate synthetic multi-dimensional data
    train_x_full, train_y_full = generate_synthetic_data(n_points, n_features)
    
    # Initialize likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    
    # Store results
    results = []
    mse_loss_list = []
    feature_importance_history = []
    
    # Expanding window cross-validation
    for day in tqdm(range(initial_train_size, len(train_x_full)), desc="Incremental Training"):
        # Update training set
        train_x = train_x_full[:day]
        train_y = train_y_full[:day]
        test_x = train_x_full[day].unsqueeze(0)
        
        # Initialize model
        model = MultiDimExactGPModel(train_x, train_y, likelihood).to(device)
        
        # Training mode
        model.train()
        likelihood.train()
        
        # Create a single optimizer with all parameters
        optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
        
        # Loss function
        mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
        
        # Training loop
        training_iter = 50
        for i in range(training_iter):
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
        
        # Get feature importance from lengthscales
        lengthscales = model.covar_module.base_kernel.lengthscale.detach().cpu().numpy().flatten()
        feature_importance = 1.0 / lengthscales  # Inverse relationship
        feature_importance = feature_importance / np.sum(feature_importance)  # Normalize
        feature_importance_history.append(feature_importance)
        
        # Print current model parameters
        if day % 10 == 0:
            print(f"\nDay {day} - Model Parameters:")
            print(f"Noise: {likelihood.noise.item():.4f}")
            print(f"Outputscale: {model.covar_module.outputscale.item():.4f}")
            print("Lengthscales:", lengthscales)
        
        # Evaluation mode
        model.eval()
        likelihood.eval()
        
        # Predict
        with torch.no_grad():
            observed_pred = likelihood(model(test_x))
            # Get prediction mean and variance
            pred_mean = observed_pred.mean.cpu().numpy().flatten()
            pred_var = observed_pred.variance.cpu().numpy().flatten()
        
        test_actual = train_y_full[day].cpu().numpy()
        
        # Calculate MSE
        mse_loss = ((pred_mean - test_actual) ** 2).mean()
        mse_loss_list.append(mse_loss)
        
        results.append({
            "day": day,
            "test_actual": test_actual,
            "pred_mean": pred_mean,
            "pred_var": pred_var,
            "feature_importance": feature_importance
        })
    
    return results, mse_loss_list, feature_importance_history

# Run the model and get results
n_features = 3  # Number of features
results, mse_loss_list, feature_importance_history = train_and_evaluate(n_features=n_features)

# Plot predictions with uncertainty
plt.figure(figsize=(12, 6))
days = [result["day"] for result in results]
test_actuals = [result["test_actual"] for result in results]
pred_means = [result["pred_mean"] for result in results]
pred_vars = [result["pred_var"] for result in results]

plt.subplot(1, 2, 1)
plt.plot(days, test_actuals, 'k*', label="Actual")
plt.plot(days, pred_means, 'b', label="Predicted Mean")
plt.fill_between(
    days, 
    [pm - 2*np.sqrt(pv) for pm, pv in zip(pred_means, pred_vars)], 
    [pm + 2*np.sqrt(pv) for pm, pv in zip(pred_means, pred_vars)], 
    color='blue', alpha=0.3,
    label="95% Confidence"
)
plt.legend()
plt.xlabel('Days')
plt.ylabel('Values')
plt.title('Multi-dimensional GPR Predictions')

# Plot MSE loss
plt.subplot(1, 2, 2)
plt.plot(days, mse_loss_list, 'r', label="MSE Loss")
plt.xlabel('Days')
plt.ylabel('MSE Loss')
plt.title('MSE Loss Over Time')
plt.legend()
plt.tight_layout()
plt.show()

# Plot feature importance over time
feature_importance_history = np.array(feature_importance_history)
plt.figure(figsize=(10, 6))
for i in range(n_features):
    plt.plot(days, feature_importance_history[:, i], label=f'Feature {i}')
plt.xlabel('Days')
plt.ylabel('Feature Importance')
plt.title('Feature Importance Evolution Over Time')
plt.legend()
plt.show()